In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, sampler, TensorDataset
from torch.utils.data import sampler

import torch.nn.functional as F

import torchvision.datasets as dset
import torchvision.transforms as T

import matplotlib.pyplot as plt

import numpy as np
import pickle
import os
from PIL import Image

from scipy.interpolate import interp2d
from scipy.interpolate import RectBivariateSpline
from scipy.ndimage.filters import gaussian_filter

import h5py

from time import time

################## LOGGING-BEGIN #########################
import torchvision.utils as vutils
from tensorboardX import SummaryWriter
################## LOGGING-END ###########################

In [ ]:
# From Assignment 2 PyTorch Notebook
USE_GPU = True

dtype = torch.float32 # we will be using float throughout

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

In [ ]:
# Adapted from https://github.com/pytorch/vision/blob/master/torchvision/datasets/cifar.py

HEIGHT_INDEX = 200
WIDTH_INDEX = 200

IMG_HEIGHT = 64
IMG_WIDTH = 64

NUM_CHANNELS_IN = 2
NUM_CHANNELS_OUT = 1

TRAIN_FILES = 10
VAL_FILES = 2
TEST_FILES = 0
YEARS_PER_FILE = 10
IMGS_PER_YEAR = 365

TRAIN_MODE = 0
VAL_MODE = 1
TEST_MODE = 2

class SR_Dataset(data.Dataset):
    """
    Args:
        root (string): Root directory of dataset where directory
            ``cifar-10-batches-py`` exists or will be saved to if download is set to True.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """
    base_folder = 'sr_data'
    elevation_file = 'dem.pkl'
    
    train_list = np.array([
        '1950s.hdf5',
        '1960s.hdf5',
        '1970s.hdf5',
        '1980s.hdf5',
        '1990s.hdf5',
        '2000s.hdf5',
        '2010s.hdf5',
        '2020s.hdf5',
        '2030s.hdf5',
        '2040s.hdf5'
    ])
    
    val_list = np.array([
        '2050s.hdf5',
        '2060s.hdf5'
    ])

    test_list = np.array([
        
    ])

    def __init__(self, root, train=TRAIN_MODE):
        
        self.root = os.path.expanduser(root)
        self.train = train  # training set or val set or test set
        
        # load elevation data
        fo_in = open(os.path.join(self.root, self.elevation_file), 'rb')
        self.elevation = pickle.load(fo_in)
        fo_in.close()
        self.elevation = self.elevation[HEIGHT_INDEX:(HEIGHT_INDEX+IMG_HEIGHT), WIDTH_INDEX:(WIDTH_INDEX+IMG_WIDTH)]
        elev_mean = np.mean(self.elevation)
        elev_var = np.var(self.elevation)
        self.elevation = (self.elevation - elev_mean) / np.sqrt(elev_var)
        h,w=self.elevation.shape
        self.elevation = self.elevation.reshape((1,h,w))

    in_mean = np.array([1.9028055e-05])
    in_var = np.array([1.5503707e-09])
    out_mean = np.array([1.902273e-05])
    out_var = np.array([2.3926674e-09])
    
    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (low_res, high_res)
        """
        t1 = time()
        file_idx = index // (YEARS_PER_FILE * IMGS_PER_YEAR)
        img_in_file = index % (YEARS_PER_FILE * IMGS_PER_YEAR)
        year_idx = img_in_file // IMGS_PER_YEAR
        idx_in_year = img_in_file % IMGS_PER_YEAR
        
        f = None
        if self.train == TRAIN_MODE:
            f = h5py.File(os.path.join(self.base_folder, self.train_list[file_idx]), 'r')
        elif self.train == VAL_MODE:
            f = h5py.File(os.path.join(self.base_folder, self.val_list[file_idx]), 'r')
        elif self.train == TEST_MODE:
            f = h5py.File(os.path.join(self.base_folder, self.test_list[file_idx]), 'r')
        
        yr = list(f.keys())[year_idx]
        pr =  f[yr]['pr'][idx_in_year]
        f.close()
        high_res = np.flip(pr[np.newaxis,:,:], axis=1)
        high_res = high_res[:, HEIGHT_INDEX:(HEIGHT_INDEX+IMG_HEIGHT), WIDTH_INDEX:(WIDTH_INDEX+IMG_WIDTH)]

        # get the input LR image from output HR image
        c,h1,w1 = high_res.shape
        blurred = np.zeros_like(high_res)
        for i in range(c):
            blurred[i,:,:] = gaussian_filter(high_res[i,:,:], 0.55)
        half_res = blurred[:, ::2, ::2]
        c,h2,w2 = half_res.shape
        x = np.arange(h2)
        y = np.arange(w2)
        xnew = np.arange(0, h2, h2/h1)
        ynew = np.arange(0, w2, w2/w1)
        low_res = np.zeros_like(high_res)
        for i in range(c):
            f = RectBivariateSpline(x, y, half_res[i, :, :])
            low_res[i, :, :] = f(xnew, ynew)
            
        low_res =  (low_res -  self.in_mean[:,np.newaxis,np.newaxis])  / np.sqrt(self.in_var[:,np.newaxis,np.newaxis])
        high_res = (high_res - self.out_mean[:,np.newaxis,np.newaxis]) / np.sqrt(self.out_var[:,np.newaxis,np.newaxis])
        
        low_res = np.concatenate((low_res, self.elevation))
        
        low_res = torch.from_numpy(low_res)
        high_res = torch.from_numpy(high_res)
        
        #print(time()-t1)
        return low_res, high_res

    def __len__(self):
        print('not useful, fake!!!')
        return 0

    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        tmp = 'train' if self.train is True else 'test'
        fmt_str += '    Split: {}\n'.format(tmp)
        fmt_str += '    Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str

In [ ]:
data_directory = '.'

batch_size = 200

sr_train = SR_Dataset(data_directory, train=TRAIN_MODE)
loader_train = DataLoader(sr_train, batch_size=batch_size, 
                          sampler=sampler.SubsetRandomSampler(range(TRAIN_FILES*YEARS_PER_FILE*IMGS_PER_YEAR)),
                         num_workers = 4)

sr_val = SR_Dataset(data_directory, train=VAL_MODE)
loader_val = DataLoader(sr_val, batch_size=batch_size, 
                          sampler=sampler.SubsetRandomSampler(range(VAL_FILES*YEARS_PER_FILE*IMGS_PER_YEAR)),
                         num_workers = 4)

#sr_val = SR_Dataset(data_directory, train=True, transform=transform, target_transform=transform)
#loader_val = DataLoader(sr_val, batch_size=batch_size, 
#                        sampler=sampler.SubsetRandomSampler(range(TRAIN_SIZE, TRAIN_SIZE+VAL_SIZE)))

#sr_test = SR_Dataset(data_directory, train=True, transform=transform)
#loader_test = DataLoader(sr_test, batch_size=batch_size)

In [ ]:
# used https://github.com/togheppi/pytorch-super-resolution-model-collection/blob/master/srcnn.py to get basic idea
class SRCNN(nn.Module):
    def __init__(self, num_channels_in, num_channels_out, hidden_channels):
        super().__init__()
        
        self.layers = nn.Sequential( # this is the original SRCNN from Dong et al. 2015
            nn.ReplicationPad2d(4),
            #nn.Conv2d(num_channels_in, hidden_channels, 9, padding=4),
            nn.Conv2d(num_channels_in, hidden_channels, 9, padding=0),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels//2, 1, padding=0),
            nn.ReLU(),
            nn.ReplicationPad2d(2),
            #nn.Conv2d(hidden_channels//2, num_channels_out, 5, padding=2)
            nn.Conv2d(hidden_channels//2, num_channels_out, 5, padding=0)
        )
        
    def forward(self, x):
        return self.layers(x)

In [ ]:
def test_SRCNN():
    x = torch.zeros((64, NUM_CHANNELS_IN, IMG_HEIGHT, IMG_WIDTH), dtype=dtype)  # minibatch size 64, feature dimension 50
    model = SRCNN(NUM_CHANNELS_IN, NUM_CHANNELS_OUT, 16)
    scores = model(x)
    print(scores.size())
test_SRCNN()

In [ ]:
def plot_epoch(x, y_pred, y):
    plt.figure(figsize=(16,12))
    plt.subplot(1,3,1)
    plt.imshow(x[0,0,:,:].cpu().detach().numpy())
    plt.subplot(1,3,2)
    plt.imshow(y_pred[0,0,:,:].cpu().detach().numpy())
    plt.subplot(1,3,3)
    plt.imshow(y[0,0,:,:].cpu().detach().numpy())
    plt.show()

In [ ]:
def plot_loss(train_loss, rmse_precip):
    tcount = np.count_nonzero(train_loss)
    
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(range(tcount), train_loss[range(tcount)])
    plt.title("SRCNN Loss")
    plt.xlabel("Iteration")
    
    plt.subplot(1,2,2)
    plt.plot(range(tcount), rmse_precip[range(tcount)])
    plt.title("SRCNN Precipitation Loss")
    plt.xlabel("Iteration")
    plt.show()

In [ ]:
################## LOGGING-BEGIN #########################
# how often to log images
save_img_every = 500
val_every = 200
print_every = 200
exp_name = 'experiments/train_SRCNN_preciponly_lr1e-4'
writer = SummaryWriter(exp_name)
################## LOGGING-END ###########################

In [ ]:
def train_it(optimizer, model, num_epochs=1):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    print("Expected num iters: ", len(loader_train)*epochs)
    train_loss = np.zeros(len(loader_train)*num_epochs+1)
    rmse_precip = np.zeros(len(loader_train)*num_epochs+1)
    iter_count = 0
    best_rmse_precip_val = 10000
    tic = time()
    for e in range(num_epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=dtype)

            y_predicted = model(x)
            loss_func = nn.MSELoss()
            loss = loss_func(y_predicted, y)
            train_loss[iter_count] = loss.item()
            
            optimizer.zero_grad()

            loss.backward()

            optimizer.step()
            
            rmse_precip[iter_count] = torch.sqrt(torch.mean((y_predicted[:, 0, :, :] - y[:, 0, :, :]).pow(2)))

            if iter_count % val_every == 0:
                loss_total = 0 
                rmse_precip_val = check_accuracy(loader_val, model)
                if rmse_precip_val < best_rmse_precip_val:
                    torch.save(model.cpu().state_dict(), 'training_SRCNN_06_05_preciponly_best_precip.pt')
                    model = model.to(device=device)
                    best_rmse_precip_val = rmse_precip_val
                    
                writer.add_scalars('Both precip RMSE',
                                      {'Train': rmse_precip[iter_count],
                                      'Val': rmse_precip_val}, iter_count)
            
            if iter_count % print_every == 0:
                print('Epoch %d, Iteration %d, loss=%.4f, RMSEp=%.4f' % (e, t, loss.item(), 
                                                                                     rmse_precip[iter_count]))
                plot_epoch(x, y_predicted, y)
                plot_loss(train_loss, rmse_precip)
                
            if iter_count % save_img_every == 0:
                # precip
                input_precip_grid = vutils.make_grid(x[0, 0, :, :])
                writer.add_image('Input precipitation', input_precip_grid, iter_count)
                output_precip_grid = vutils.make_grid(y_predicted[0, 0, :, :])
                writer.add_image('Output precipitation', output_precip_grid, iter_count)
                true_precip_grid = vutils.make_grid(y[0, 0, :, :])
                writer.add_image('True precipitation', true_precip_grid, iter_count)
            
            iter_count += 1
                
            del x, y, loss
        torch.save(model.cpu().state_dict(), 'training_SRCNN_06_05_preciponly_checkpoint.pt')
        model = model.to(device=device)
        print('Epoch ', e, ' complete!')

In [ ]:
def check_baseline_accuracy(loader):
    #if loader.dataset.train == TRAIN_MODE:
    #    print('Checking accuracy on train set')
    #elif loader.dataset.train == VAL_MODE:
    #    print('Checking accuracy on validation set')
    #elif loader.dataset.train == TEST_MODE:
    #    print('Checking accuracy on test set')
    count, rmse_precip_x = 0, 0, 
    with torch.no_grad():
        for x, y in loader:
            y = y.to(device=device, dtype=dtype)
            
            # Normalize x to be in -1 to 1 for purpose of comparing with high res data in same range
            # Turn it into a numpy array
            x_np = x.numpy()
            x_min = np.amin(x_np, axis=(2,3))[:, :, np.newaxis, np.newaxis]
            x_max = np.amax(x_np, axis=(2,3))[:, :, np.newaxis, np.newaxis]
            is_nan = np.int((x_min == x_max).any())
            eps = 1e-9
            x_norm_np = (x_np - x_min) / ((x_max - x_min + is_nan*eps) / 2) - 1
            
            x_norm = torch.from_numpy(x_norm_np)
            x_norm = x_norm.to(device=device, dtype=dtype)
            
            rmse_precip_x += torch.sqrt(torch.mean((x_norm[:,0,:,:]-y[:,0,:,:]).pow(2)))
            count += 1
            
        rmse_precip_x /= count
        print('RMSEs: \tInput precip: %.3f\n' % 
              (rmse_precip_x))
        
    return rmse_precip_x

def check_accuracy(loader, model):
    model = model.to(device=device)
    model.eval() # set model to evaluation mode
    count, rmse_precip = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype) # move to device, e.g. GPU
            y = y.to(device=device, dtype=dtype)
            
            y_predicted = model(x)
            rmse_precip += torch.sqrt(torch.mean((y_predicted[:,0,:,:]-y[:,0,:,:]).pow(2)))
            count += 1
            
        rmse_precip /= count

    return rmse_precip

In [ ]:
learning_rate = 1e-4

epochs=20

check_baseline_accuracy(loader_val)
model = SRCNN(NUM_CHANNELS_IN, NUM_CHANNELS_OUT, 64)
#model.load_state_dict(torch.load('training_SRCNN_checkpoint_crop.pt'))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_it(optimizer, model, num_epochs=epochs)